# Memento
> Without violating encapsuation, capture and externalize an object's internal state so that the object can be restored to this state later.

# Problem
Lets say I have an "expensive" object, i.e., an object that took a lot of time/effort to get to its current state. Examples of such objects are the current state of a game, an ML model that is far into its training, etc. How can I save its current state and destroy the current object and restore its saved state again? E.g., saving and restoring checkpoints in a game.

In the code below the way the game is saved and restored is really bad because now the `gameplay` module knows about the internal state of the game. How can I acheive this functionality but without breaking encapsulation of `MyCoolGame`?

In [6]:
import random

class MyCoolGame:
    def __init__(self):
        self._state = 0
        
    def play(self):
        # After much time and effort
        print(f"Starting state {self._state}")
        self._state += random.randint(10, 1000)
        print(f"Ending state {self._state}")
        
def gameplay():
    game1 = MyCoolGame()
    game1.play()
    game1.play()
    game_state = game1._state
    # save the game state on disk
    
    game2 = MyCoolGame()
    game2._state = game_state
    game2.play()

In [8]:
gameplay()

Starting state 0
Ending state 997
Starting state 997
Ending state 1713
Starting state 1713
Ending state 1744


## Solution
At its simplest implement `save` and `restore` methods in the `MyCoolGame` class. The client calling these methods is given an opaque binary checkpoint object that it can give an new instance of the game to restore.

In [9]:
class Checkpoint:
    def __init__(self):
        self.state = None
        self.filename = None
        
    @classmethod
    def dump(cls, game, filename):
        obj = cls()
        obj.state = game._state
        # Persist the game state in the filename
        obj.filename = filename
        return obj
    
    @classmethod
    def load(cls, game, filename):
        obj = cls()
        # Load the state from the filename
        state = random.randint(10, 1000)
        obj.state = state
        obj.filename = filename
        game._state = state
    
class MyCoolGame:
    def __init__(self):
        self._state = 0
        
    def play(self):
        # After much time and effort
        print(f"Starting state {self._state}")
        self._state += random.randint(10, 1000)
        print(f"Ending state {self._state}")
        
    def checkpoint(self):
        # Generate a filename
        filename = "path/to/checkpoint"
        ckpt = Checkpoint.dump(self, filename)
        return filename
    
    def restore(self, filename):
        Checkpoint.load(self, filename)

def gameplay():
    game1 = MyCoolGame()
    game1.play()
    game1.play()
    fn = game1.checkpoint()
    # Save the filename somewhere
    
    game2 = MyCoolGame()
    game2.restore(fn)
    game2.play()
    
gameplay()

Starting state 0
Ending state 398
Starting state 398
Ending state 424
Starting state 158
Ending state 1053
